# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [2]:
import matplotlib.pyplot as plt
#import mplleaflet
import pandas as pd
import numpy as np
import matplotlib.dates as mdates

%matplotlib inline

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c = 'r', alpha = 0.5, s = 100)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

FileNotFoundError: File b'data/C2A2_data/BinSize_d400.csv' does not exist

In [3]:
# Importing Data and trimming the unwanted part

data = pd.read_csv("data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv")
data = data.drop("ID", axis=1)
data['Date'] = pd.to_datetime(data['Date'])
data = data[data["Date"].map(lambda d: not ((d.month == 2) and (d.day == 29)))]


FileNotFoundError: File b'data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv' does not exist

In [3]:
# Handling data

date_index = pd.DatetimeIndex(data['Date'])
data['Month'] = date_index.month
data['Day'] = date_index.day

data_2015 = data[date_index>='2015-01-01']
data_early = data[date_index<='2015-01-01']

data_max = (data_early[data_early['Element'] == 'TMAX'].groupby(['Month', 'Day'])['Data_Value'].max())/10
data_min = (data_early[data_early['Element'] == 'TMIN'].groupby(['Month', 'Day'])['Data_Value'].min())/10

data_max_2015 = (data_2015[data_2015['Element'] == 'TMAX'].groupby(['Month', 'Day'])['Data_Value'].max())/10
data_min_2015 = (data_2015[data_2015['Element'] == 'TMIN'].groupby(['Month', 'Day'])['Data_Value'].min())/10

# data = complete data
# data_2015 = data only for 2015
# data_early = data for 2005 - 2014
# data_max = max temp for each day and month upto 2014

# When the records were broken
max_rec_broken = (data_max_2015 > data_max).as_matrix()
min_rec_broken = (data_min_2015 < data_min).as_matrix()

observation_axis = np.arange('2015-01-01', '2016-01-01', dtype='datetime64[D]')
observation_axis = np.array(list(map(pd.to_datetime, observation_axis)))


In [4]:
# Plotting

plt.figure(figsize=(16,8))
plt.title('2015 temperature records against the previous 10 years (2005 - 2014) ')

# Converting x axis to months format
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())

# plotting 2005 - 2014 data
plt.plot(observation_axis, data_min.as_matrix(), label = 'Past Daily Minimum', c = 'blue', alpha = 0.5)
plt.plot(observation_axis, data_max.as_matrix(), label = 'Past Daily Maximum', c = 'orange')

#plotting 2015 data
plt.scatter(observation_axis[max_rec_broken], data_max_2015[max_rec_broken].as_matrix(), label = 'record breaking for 2015',
           marker = '+', c = 'red')
plt.scatter(observation_axis[min_rec_broken], data_min_2015[min_rec_broken].as_matrix(), label = 'record breaking for 2015',
           marker = '_', c = 'purple')
plt.gca().fill_between(observation_axis, data_min, data_max, alpha = 0.8)
plt.grid(linestyle = ':', alpha = 0.6)
plt.xlabel('Months --->')
plt.ylabel('Temp in Celsius --->')

for spine in plt.gca().spines.values():
    spine.set_visible(False)

plt.legend()